# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: Lora
* Model: GPT2
* Evaluation approach:accuracy 
* Fine-tuning dataset: imdb

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [3]:
import numpy as np
import pandas as pd
import os
from datasets import load_dataset
from transformers import AutoTokenizer, GPT2Config,GPT2TokenizerFast, AutoModelForSequenceClassification,GPT2Tokenizer
from transformers import GPT2ForSequenceClassification, Trainer, TrainingArguments
from collections import Counter
from transformers import DataCollatorForLanguageModeling
from transformers import DataCollatorWithPadding


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


/opt/conda/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cpu.so: undefined symbol: cadam32bit_grad_fp32


In [4]:
#!pip install accelerate.

In [5]:
#!pip install bitsandbytes.
#!pip install -i https://test.pypi.org/simple/ bitsandbytes

### Load the IMDb dataset from Hugging Face 

The IMDb dataset is a widely used benchmark dataset for binary sentiment classification tasks in natural language processing (NLP). It consists of 50,000 movie reviews taken from the Internet Movie Database (IMDb), a popular online database of information related to films, television programs, home videos, video games, and streaming content. These reviews are evenly split into two sets: 25,000 reviews for training and 25,000 for testing. Each set contains an equal number of positive and negative reviews, making it a balanced dataset.

Positive reviews are those with a rating of 7 or higher out of 10, and negative reviews have a rating of 4 or lower; reviews with neutral ratings (5-6) are not included in the dataset. This clear demarcation helps in training models to distinguish between positive and negative sentiments effectively.

The reviews are preprocessed and each one is encoded as plain text. Labels are provided indicating the sentiment of each review, where a label of 1 typically represents a positive sentiment and 0 represents a negative sentiment. This dataset is particularly useful for training and evaluating models on sentiment analysis tasks because it contains real-world data with a wide range of vocabulary, idiomatic expressions, and varying lengths of text, making it a comprehensive resource for training machine learning models to understand human sentiment.

The IMDb dataset is not only a benchmark for sentiment analysis but also serves as a valuable resource for exploring text classification, natural language understanding, and the effectiveness of various NLP models and techniques in discerning subjective information from text.







In [6]:
# Import the datasets and transformers packages

dataset = load_dataset("imdb")


Generating unsupervised split: 100%|██████████| 50000/50000 [00:00<00:00, 239359.65 examples/s]


In [7]:
#from transformers import AutoModelForCausalLM
#model = AutoModelForCausalLM.from_pretrained("gpt2")

In [8]:
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 25000
 })}

The dataset is sizable, prompting us to experiment with various split sizes to enhance accuracy. Achieving a perfect balance—50% positive and 50% negative responses—was only feasible with 2000 rows for both the training and testing datasets. However, due to GPU memory constraints, we downsized to 500 rows.






In [9]:
# Load the train and test splits of the imdb dataset
splits = ["train", "test"]
ds = {split: ds for split, ds in zip(splits, load_dataset("imdb", split=splits))}

# Thin out the dataset to make it run faster for this example
for split in splits:
    ds[split] = ds[split].shuffle(seed=42).select(range(500))

# Show the dataset
ds

{'train': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'label'],
     num_rows: 500
 })}

In [10]:
# Dataset with 'train' and 'test' splits
train_labels = ds['train']['label']
test_labels = ds['test']['label']

# Count the labels in both splits
train_label_counts = Counter(train_labels)
test_label_counts = Counter(test_labels)

# Calculate proportions
train_label_proportions = {label: count / len(train_labels) for label, count in train_label_counts.items()}
test_label_proportions = {label: count / len(test_labels) for label, count in test_label_counts.items()}

# Print the statistics
print("Train Label Counts:", train_label_counts)
print("Train Label Proportions:", train_label_proportions)
print("Test Label Counts:", test_label_counts)
print("Test Label Proportions:", test_label_proportions)


Train Label Counts: Counter({0: 254, 1: 246})
Train Label Proportions: {1: 0.492, 0: 0.508}
Test Label Counts: Counter({0: 254, 1: 246})
Test Label Proportions: {1: 0.492, 0: 0.508}


In [11]:
# Function to rename 'label' column to 'labels' for compatibility with transformer library
def rename_label_to_labels(example):
    example['labels'] = example['label']
    return example

# Rename the column in both the train and test datasets
for split in ['train', 'test']:
    ds[split] = ds[split].map(rename_label_to_labels, batched=True)

# Remove the old 'label' column if you want to clean up
for split in ['train', 'test']:
    ds[split] = ds[split].remove_columns(['label'])


Map: 100%|██████████| 500/500 [00:00<00:00, 66462.32 examples/s]


In [12]:
ds

{'train': Dataset({
     features: ['text', 'labels'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'labels'],
     num_rows: 500
 })}

In [13]:
#Exploration of the splits
for split in splits:
    print(f"First few entries in {split} dataset:")
    for i in range(5):  # Adjust the range as needed
        print(ds[split][i]['labels'])
    print("\n")


First few entries in train dataset:
1
1
0
1
0


First few entries in test dataset:
1
1
0
1
0




In [14]:
# Dataset to be converted to a pandas DataFrame
for split in splits:
    df = pd.DataFrame(ds[split])
    print(f"Unique values in 'sentiment' column for {split} dataset:")
    print(df['labels'].unique())
    print("\n")


Unique values in 'sentiment' column for train dataset:
[1 0]


Unique values in 'sentiment' column for test dataset:
[1 0]




### Preprocessing and Loading the Tokenizer from Transformer Lybrary

In [15]:
# Load the GPT-2 tokenizer
#tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer = GPT2TokenizerFast.from_pretrained('gpt2')
tokenizer.padding_side = "right"

# Set the pad token to the EOS token (End Of Sentence token)
tokenizer.pad_token = tokenizer.eos_token



vocab.json: 100%|██████████| 1.04M/1.04M [00:00<00:00, 16.3MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 6.49MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 48.5MB/s]
config.json: 100%|██████████| 665/665 [00:00<00:00, 2.85MB/s]


In [16]:
#from transformers import AutoModelForCausalLM
#model = AutoModelForCausalLM.from_pretrained("gpt2")

### Load the GPT2 Pretrained Model from Hugging Face

In [17]:
# Update the GPT-2 model configuration to recognize the new padding token and the text task classification
config = GPT2Config.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    pad_token_id=tokenizer.eos_token_id  # Add pad token id to the model config
)

# Load the model with the updated configuration for sequence classification
model = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config)

# Freeze all the parameters of the base model
for param in model.base_model.parameters():
    param.requires_grad = False

# The model is now ready with the appropriate settings for padding token
model.config.pad_token_id = tokenizer.pad_token_id

model.safetensors: 100%|██████████| 548M/548M [00:02<00:00, 222MB/s] 
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [18]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=768, out_features=2, bias=False)
)

### Dysplay Sample Tokens

In [19]:
def preprocess_function(examples):
    """Preprocess the dataset by returning tokenized examples with labels."""
    # Tokenize the text
    tokenized_ds = tokenizer(examples["text"], padding=True, truncation=True, max_length=512)
    #tokenized_ds = dataset.map(tokenize_function, batched=True)
    # Add labels (ensure 'sentiment' column exists and contains appropriate labels)
    tokenized_ds["labels"] = examples["labels"]
    return tokenized_ds

# Apply the preprocessing function to each split
tokenized_ds = {split: ds[split].map(preprocess_function, batched=True) for split in splits}

# Show the first example of the tokenized training set
print(tokenized_ds["train"][0])



Map: 100%|██████████| 500/500 [00:00<00:00, 1268.17 examples/s]

{'text': 'There is no relation at all between Fortier and Profiler but the fact that both are police series about violent crimes. Profiler looks crispy, Fortier looks classic. Profiler plots are quite simple. Fortier\'s plot are far more complicated... Fortier looks more like Prime Suspect, if we have to spot similarities... The main character is weak and weirdo, but have "clairvoyance". People like to compare, to judge, to evaluate. How about just enjoying? Funny thing too, people writing Fortier looks American but, on the other hand, arguing they prefer American series (!!!). Maybe it\'s the language, or the spirit, but I think this series is more English than American. By the way, the actors are really good and funny. The acting is not superficial at all...', 'labels': 1, 'input_ids': [1858, 318, 645, 8695, 379, 477, 1022, 6401, 959, 290, 4415, 5329, 475, 262, 1109, 326, 1111, 389, 1644, 2168, 546, 6590, 6741, 13, 4415, 5329, 3073, 42807, 11, 6401, 959, 3073, 6833, 13, 4415, 5329, 2

In [20]:
from transformers import GPT2Tokenizer, DataCollatorForLanguageModeling


In [21]:
# Checking the first few samples of the training and testing datasets
for split in ['train', 'test']:
    for i in range(3):  # Check first 3 samples
        print(f"{split.upper()} Sample {i}:")
        print("Input Length:", len(tokenized_ds[split][i]['input_ids']))
        print("Labels:", tokenized_ds[split][i]['labels'] if 'labels' in tokenized_ds[split][i] else "No labels")
        print("\n")


TRAIN Sample 0:
Input Length: 512
Labels: 1


TRAIN Sample 1:
Input Length: 512
Labels: 1


TRAIN Sample 2:
Input Length: 512
Labels: 0


TEST Sample 0:
Input Length: 512
Labels: 1


TEST Sample 1:
Input Length: 512
Labels: 1


TEST Sample 2:
Input Length: 512
Labels: 0




### Define Metrics to evaluate the Model

When training a large language model (LLM) for binary text classification, such as distinguishing between positive and negative sentiments in movie reviews, relying solely on accuracy as a performance metric can be misleading. This is particularly true in scenarios where the dataset is imbalanced, meaning one class significantly outnumbers the other. In such cases, a model might achieve high accuracy by simply predicting the majority class for all instances, but this doesn't necessarily indicate that the model is performing well in identifying the nuances between classes.

- **Precision and Recall**:  
Precision (the ratio of true positive predictions to all positive predictions) and recall (the ratio of true positive predictions to all actual positives) are particularly useful when dealing with imbalanced datasets. Precision focuses on the purity of positive predictions, while recall emphasizes the completeness of positive detection.

- **F1 Score**:  
The F1 score is the harmonic mean of precision and recall, providing a single metric that balances both. An F1 score is particularly useful when you need to balance precision and recall, and there's not a clear preference for one over the other. It is more informative than accuracy, especially in the context of imbalanced classes.


In [22]:
def compute_metrics(eval_pred):
    """
    Compute the accuracy of the model's predictions.

    This function calculates the accuracy metric for evaluating the model's performance. 
    The decision to utilize accuracy as the sole evaluation metric was influenced by constraints encountered within the Udacity workspace environment. 
    Specifically, issues importing the scikit-learn library prevented the application of the F1 score as an additional metric. 
    While the F1 score was considered and applied on a local machine, the lack of GPU resources available outside of the Udacity workspace made comprehensive evaluation impractical. 
    Consequently, to maintain project continuity and efficiency within the available infrastructure, it was decided to solely rely on accuracy for project completion.

    Parameters:
    - eval_pred: A tuple of (predictions, labels), where predictions are the logits returned by the model, and labels are the ground truth labels.

    Returns:
    - A dictionary containing the accuracy score under the key "accuracy".
    """
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}


In [23]:
#from datasets import load_metric

#accuracy_metric = load_metric("accuracy")

In [24]:
# Freeze all the parameters of the base model
#for param in model.base_model.parameters():
    #param.requires_grad = False


In [25]:
from transformers import DataCollatorWithPadding
"""
This script initializes a `DataCollatorWithPadding` object from the Hugging Face Transformers library. The data collator is used to dynamically pad the inputs to the maximum length in a batch, ensuring consistent input sizes for model training or evaluation. This is particularly useful when working with variable-length sequences in NLP tasks, such as text classification, where input samples can vary significantly in length.

The `DataCollatorWithPadding` takes a tokenizer as an argument, which is used to pad the inputs based on the tokenizer's padding token. This ensures that the padding is consistent with the tokenizer's specifications, allowing for seamless integration with models pre-trained using the same tokenizer.

Parameters:
- tokenizer: The tokenizer instance specific to the pre-trained model being used. This tokenizer is responsible for converting text inputs into the numerical format expected by the model, as well as applying padding.

Usage:
To use the `data_collator` with a DataLoader in PyTorch, simply pass it as the `collate_fn` argument. This ensures that each batch processed by the DataLoader is dynamically padded to the longest sequence in that batch, optimizing computational efficiency and model performance.
"""


data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


### Testing the Model for Sentiment Analisis task classification before PEFT Lora

In [26]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding


In [ ]:
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysisft",
        evaluation_strategy='epoch',
        learning_rate=2e-4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir='./logs',
    ),
    train_dataset=tokenized_ds["train"].select(range(100)),  # Using a subset for faster training
    eval_dataset=tokenized_ds["test"].select(range(100)),  # Using a subset for faster evaluation
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


In [22]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments

#tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
#model = GPT2ForSequenceClassification.from_pretrained('gpt2', num_labels=2)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

from transformers import Trainer, TrainingArguments, DataCollatorWithPadding

trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir="./data/sentiment_analysisft",
        evaluation_strategy='epoch',
        learning_rate=2e-4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir='./logs',
    ),
    train_dataset=tokenized_ds["train"].select(range(100)),  # Using a subset for faster training
    eval_dataset=tokenized_ds["test"].select(range(100)),  # Using a subset for faster evaluation
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

trainer.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.862696,0.530000
2,No log,0.647147,0.690000
3,No log,0.676570,0.590000
4,No log,0.639882,0.700000
5,No log,0.634058,0.670000
6,No log,0.635922,0.680000
7,No log,0.653302,0.670000
8,No log,0.639679,0.690000
9,No log,0.636725,0.680000
10,No log,0.636613,0.680000


TrainOutput(global_step=130, training_loss=0.7741219153771034, metrics={'train_runtime': 87.7819, 'train_samples_per_second': 11.392, 'train_steps_per_second': 1.481, 'total_flos': 261296750592000.0, 'train_loss': 0.7741219153771034, 'epoch': 10.0})

In [22]:
#from transformers import DataCollatorWithPadding

#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding='max_length')


In [2]:
# Show the performance of the model on the test set
from transformers import Trainer
# What do you think the evaluation accuracy will be?
trainer.evaluate()

In [28]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

checkpoint = "./data/sentiment_analysis/checkpoint-25"#./data/sentiment_analysis/checkpoint-25#./data/sentiment_analysisft/checkpoint-75
tokenizer 
= AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# Print model and tokenizer details for inspection
print("Model Configuration:", model.config)
print("Tokenizer Configuration:", tokenizer.backend_tokenizer)


Model Configuration: GPT2Config {
  "_name_or_path": "./data/sentiment_analysis/checkpoint-25",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2ForSequenceClassification"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 50256,
  "problem_type": "single_label_classification",
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

In [29]:
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification, AutoTokenizer

# Assuming your checkpoint is in "path/to/your/checkpoint"
checkpoint = "./data/sentiment_analysis/checkpoint-25"
tokenizer = GPT2TokenizerFast.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
tokenizer.padding_side = "right"
tokenizer.pad_token = tokenizer.eos_token


# Define dummy `TrainingArguments` for inference
training_args = TrainingArguments(
    output_dir="./results",  # where to save model predictions
    do_predict=True,  # set to True for prediction
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    tokenizer=tokenizer,
)

# Assuming `tokenized_ds` is your dataset prepared for the model
#predictions = trainer.predict(tokenized_ds["test"])

# `predictions` now contains your model outputs

# Select the first 10 examples for prediction
subset = tokenized_ds["test"].select(range(10))

# Use the Trainer to predict on the selected subset
predictions = trainer.predict(subset)

# Now, `predictions` contains the outputs for the first 10 examples


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [30]:
predictions

PredictionOutput(predictions=array([[ 7.437575,  8.655436],
       [ 8.306036, 10.392733],
       [ 9.365354,  9.832128],
       [ 5.165733,  7.09861 ],
       [ 9.42747 ,  9.55782 ],
       [ 9.358664,  9.778226],
       [ 8.487845,  9.956489],
       [12.205111, 10.318361],
       [ 8.701191,  7.485458],
       [ 5.480114,  8.683409]], dtype=float32), label_ids=array([1, 1, 0, 1, 0, 1, 1, 0, 0, 1]), metrics={'test_loss': 0.33783215284347534, 'test_runtime': 1.0389, 'test_samples_per_second': 9.626, 'test_steps_per_second': 1.925})

### View Results from the model before PEFT Lora

In [31]:
# Select the first 10 examples from the tokenized test dataset
subset = tokenized_ds["test"].select(range(10))

# Convert the subset to a pandas DataFrame
df = pd.DataFrame({'text': [example['text'] for example in subset], 'labels': [example['labels'] for example in subset]})

# Use the trained model to make predictions on the subset
predictions = trainer.predict(subset)

# Convert raw prediction logits to discrete labels (0 or 1)
df["predicted_label"] = np.argmax(predictions.predictions, axis=1)

# Display the DataFrame
df


,text,labels,predicted_label
0,<br /><br />When I unsuspectedly rented A Thou...,1,1
1,This is the latest entry in the long series of...,1,1
2,This movie was so frustrating. Everything seem...,0,1
3,"I was truly and wonderfully surprised at ""O' B...",1,1
4,This movie spends most of its time preaching t...,0,1
5,After a very long time Marathi cinema has come...,1,1
6,"This is a really sad, and touching movie! It d...",1,1
7,Don't pay any attention to the rave reviews of...,0,0
8,Porn legend Gregory Dark directs this cheesy h...,0,0
9,This was a great movie. Something not only for...,1,1


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

## PEFT Lora

In [28]:
#!pip install peft

In [31]:
from peft import LoraConfig, TaskType

In [31]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model


In [ ]:
"""
Configure the LoRA (Low-Rank Adaptation) parameters for fine-tuning a GPT-2 model on a sequence classification task. LoRA is designed to efficiently adapt pre-trained models with minimal additional parameters, focusing on modifying only the attention and projection layers for task-specific adjustments.

Parameters:
- r (int): Rank of the low-rank approximation. This parameter determines the size of the trainable matrices introduced by LoRA. A lower rank limits the capacity of the adaptation, while a higher rank increases it, potentially improving performance at the cost of more parameters.
- lora_alpha (int): Alpha value for the LoRA regularization term. This controls the scaling of the low-rank matrices and can affect the extent to which LoRA modifies the pre-trained model's weights. A higher alpha increases the impact of the LoRA adjustments.
- lora_dropout (float): Dropout rate applied to the LoRA layers. This helps prevent overfitting by randomly zeroing out some of the elements in the LoRA matrices during training.
- bias (str): Bias type to be used in the adapted layers. Options typically include 'none', 'zero', or 'learnable'. 'None' indicates no bias, while 'learnable' allows the bias to be updated during training.
- target_modules (list): List of module names within the model to apply LoRA to. Common targets include the attention ('c_attn') and projection ('c_proj') layers of Transformer models, as these are key to adapting the model's focus and outputs.
- task_type (str): Type of the task for which the model is being fine-tuned. This example uses "TaskType.SEQ_CLS" to indicate a sequence classification task, guiding the adaptation process towards optimizing for this specific task.
- fan_in_fan_out (bool): Whether to adjust the initialization of the low-rank matrices based on the fan-in and fan-out of the original model weights. This can help maintain the initialization scale, potentially leading to more stable training.

This configuration is specifically tailored for enhancing a pre-trained GPT-2 model's performance on a text classification task using LoRA, by introducing and tuning a minimal set of additional parameters.

Usage:
Pass this configuration object to the LoRA fine-tuning process to apply these settings to the specified GPT-2 model. This approach allows for efficient, task-specific adaptation of large language models with minimal computational overhead.
"""
config = LoraConfig(
    r=8,  # Rank of the low-rank approximation
    lora_alpha=32,  # Alpha value for the LoRA regularization term
    lora_dropout=0.01,  # Dropout rate
    bias="none",  # Bias type
    target_modules=['c_attn', 'c_proj'],
    task_type="TaskType.SEQ_CLS",
    fan_in_fan_out=True,  # Setting fan_in_fan_out if it's a supported parameter
)


In [31]:
from peft import get_peft_model
lora_model = get_peft_model(model, config)

In [32]:
# Before setting the format to 'torch'
print("Before setting to torch:")
print("Type:", type(tokenized_ds["train"][0]['input_ids']))
# If it's a NumPy array or a list, you can also print its first element's type and length
if isinstance(tokenized_ds["train"][0]['input_ids'], (list, np.ndarray)):
    print("Element type:", type(tokenized_ds["train"][0]['input_ids'][0]))
    print("Length:", len(tokenized_ds["train"][0]['input_ids']))

# Setting the format to 'torch'
tokenized_ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_ds["test"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# After setting the format to 'torch'
print("\nAfter setting to torch:")
tensor = tokenized_ds["train"][0]['input_ids']
print("Type:", type(tensor))
print("Tensor dtype:", tensor.dtype)  # Data type of the tensor
print("Tensor shape:", tensor.shape)  # Shape of the tensor


Before setting to torch:
Type: <class 'list'>
Element type: <class 'int'>
Length: 512

After setting to torch:
Type: <class 'torch.Tensor'>
Tensor dtype: torch.int64
Tensor shape: torch.Size([512])


In [33]:
from torch.utils.data import DataLoader

# Creating a DataLoader for the training dataset
train_dataloader = DataLoader(tokenized_ds['train'], batch_size=8)

# Fetch the first batch
first_batch = next(iter(train_dataloader))

# Inspect the 'labels' of the first batch
if 'labels' in first_batch:
    labels_batch = first_batch['labels']
    print("Batch Labels - Type:", type(labels_batch))
    print("Batch Labels - Dtype:", labels_batch.dtype)
    print("Batch Labels - Shape:", labels_batch.shape)  # Should be [8] for a batch size of 8
else:
    print("The batch does not contain 'labels'.")

Batch Labels - Type: <class 'torch.Tensor'>
Batch Labels - Dtype: torch.int64
Batch Labels - Shape: torch.Size([8])


In [34]:
# After setting the dataset format to 'torch'
# Inspecting the 'attention_mask' tensor for the first example in the training set

# Check if 'attention_mask' exists in the dataset
if 'attention_mask' in tokenized_ds['train'].features:
    attention_mask_tensor = tokenized_ds['train'][0]['attention_mask']

    print("Attention Mask - Type:", type(attention_mask_tensor))
    print("Attention Mask - Dtype:", attention_mask_tensor.dtype)
    print("Attention Mask - Shape:", attention_mask_tensor.shape)
else:
    print("The dataset does not contain an 'attention_mask' field.")


Attention Mask - Type: <class 'torch.Tensor'>
Attention Mask - Dtype: torch.int64
Attention Mask - Shape: torch.Size([512])


### Procedure for Implementing PEFT LoRA on GPT-2 for Sentiment Analysis Using Custom Training Loops

In the pursuit of developing a sentiment analysis model leveraging the GPT-2 architecture with Parameter-Efficient Fine-Tuning (PEFT) via Low-Rank Adaptation (LoRA), we encountered operational challenges with the Hugging Face Transformers' Trainer module. Consequently, we opted for a more hands-on approach using custom training loops within PyTorch. We could consider benefits of this approach :

Customization and Control: Directly working with PyTorch provided the necessary granularity of control over every aspect of the training process, from the forward pass and loss calculation to the nuanced application of backpropagation. This level of detail was essential for integrating LoRA effectively.

Optimization Strategies: The custom training approach allowed for the implementation of specialized optimization strategies tailored to the unique requirements of sentiment analysis, facilitating more effective fine-tuning of the model.

Model Architecture Adjustments: It enabled precise adjustments to the model's architecture, ensuring that LoRA modifications were accurately applied and aligned with the model's sentiment analysis objectives.

Despite these advantages, transitioning from a PyTorch-based training regimen to leveraging Hugging Face's Transformers library for inference presented certain drawbacks. Notably, biases in the inference results were observed, which may be attributed to the following:

Model Configuration Mismatch: Potential discrepancies between the model's training configuration in PyTorch and its configuration when loaded for inference using Transformers. Such mismatches can subtly influence model performance and behavior.

Tokenizer Consistency: Differences in tokenization between training and inference phases. Even minor variations in tokenization can lead to significant shifts in model output.

Loss of Training Specifics: The Trainer module abstracts away many details of the training process. When moving to inference, certain nuances and optimizations of the custom training setup may not be fully captured, possibly leading to unexpected model behavior.

In summary, while the direct use of PyTorch for training was necessitated by the need for deep customization and control, it also introduced probably some of observed inconsistencies when transitioning to the Transformers library for inference. 

In [ ]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm

In [36]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"eval_accuracy": (predictions == labels).mean()}


In [37]:
train_dataloader = DataLoader(tokenized_ds["train"].select(range(100)), 
                              batch_size=8, 
                              collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
eval_dataloader = DataLoader(tokenized_ds["test"].select(range(100)), 
                             batch_size=8, 
                             collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))


In [38]:
optimizer = AdamW(lora_model.parameters(), lr=2e-4)

In [39]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Linear(
            in_features=768, out_features=2304, bias=True
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.01, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=768, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=8, out_features=2304, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (c_proj): Linear(
            in_features=768, out_features=768, bias=True
            (lora_dropout): ModuleDict(
  

In [40]:
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader
from transformers import AdamW
import torch
from tqdm.auto import tqdm
import numpy as np
import os

for epoch in range(10):  # Adjust the number of epochs
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Epoch {epoch} train loss: {avg_train_loss}")

    # Evaluation Step
    model.eval()
    all_predictions, all_labels = [], []
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(batch["labels"].cpu().numpy())

    eval_accuracy = np.mean(np.array(all_predictions) == np.array(all_labels))
    # Add more metrics as needed
    print(f"Epoch {epoch} eval accuracy: {eval_accuracy}")

    # Saving model checkpoint at the end of each epoch
    checkpoint_path = os.path.join('./datapetflora/sentiment_analysis/', f'checkpoint-{epoch}')
    os.makedirs(checkpoint_path, exist_ok=True)
    model.save_pretrained(checkpoint_path)
    tokenizer.save_pretrained(checkpoint_path)

# Optionally, save the final model at the end of training
final_model_path = os.path.join('./datapetflora/sentiment_analysis/', 'lora_model_GPT2')
os.makedirs(final_model_path, exist_ok=True)
model.save_pretrained(final_model_path)
tokenizer.save_pretrained(final_model_path)

# Optionally, save the final model at the end of training
#if epoch == total_epochs - 1:  # Check if it's the last epoch
 #   final_model_path = os.path.join('./results/sentiment_analysis/', 'model_final.pth')
  #  os.makedirs(os.path.dirname(final_model_path), exist_ok=True)  # Ensure the directory exists
    
    # Save not just the model, but also the optimizer state, and (optionally) the scheduler state
   # torch.save({
    #    'model_state_dict': model.state_dict(),
     #   'optimizer_state_dict': optimizer.state_dict(),
        # Include scheduler state if you have one
        # 'scheduler_state_dict': scheduler.state_dict(),
    #}, final_model_path)


100%|██████████| 13/13 [00:09<00:00,  1.35it/s]


Epoch 0 train loss: 4.12877920957712
Epoch 0 eval accuracy: 0.49


100%|██████████| 13/13 [00:09<00:00,  1.44it/s]


Epoch 1 train loss: 1.2221776338723989
Epoch 1 eval accuracy: 0.48


100%|██████████| 13/13 [00:09<00:00,  1.41it/s]


...

100%|██████████| 13/13 [00:09<00:00,  1.41it/s]


Epoch 8 train loss: 0.2801224675316077
Epoch 8 eval accuracy: 0.66


100%|██████████| 13/13 [00:09<00:00,  1.40it/s]


Epoch 9 train loss: 0.25279098691848606
Epoch 9 eval accuracy: 0.64


('./datapetflora/sentiment_analysis/lora_model_GPT2/tokenizer_config.json',
 './datapetflora/sentiment_analysis/lora_model_GPT2/special_tokens_map.json',
 './datapetflora/sentiment_analysis/lora_model_GPT2/vocab.json',
 './datapetflora/sentiment_analysis/lora_model_GPT2/merges.txt',
 './datapetflora/sentiment_analysis/lora_model_GPT2/added_tokens.json',
 './datapetflora/sentiment_analysis/lora_model_GPT2/tokenizer.json')

In [33]:
#data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="pt")

### Quantization and Lora --QLoRa

Implementing Quantization and LoRA (QLoRa) in large language models (LLMs) presents significant benefits for practical applications, particularly in enhancing the economic model of deploying AI solutions. Quantization reduces the model's memory footprint and speeds up inference by using lower-precision arithmetic, making LLMs more accessible on devices with limited computational resources. LoRA, on the other hand, allows for efficient fine-tuning of pre-trained models with minimal additional parameters, maintaining performance while reducing the computational cost of adapting models to specific tasks. Together, QLoRa can significantly lower the barriers to deploying state-of-the-art LLMs across various sectors, reducing costs and enabling real-time, on-device AI applications. This approach democratizes access to powerful AI technologies, fostering innovation and expanding the scope of AI's economic impact by making it feasible for a broader range of businesses to integrate advanced natural language processing capabilities into their operations.

In [34]:
from peft import LoraConfig, get_peft_model

In [32]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm

In [33]:
lora_config = LoraConfig(
    r=8,  # Rank of the low-rank approximation
    lora_alpha=32,  # Alpha value for the LoRA regularization term
    lora_dropout=0.01,  # Dropout rate
    bias="none",  # Bias type
    target_modules=['c_attn', 'c_proj'],
    task_type="TaskType.SEQ_CLS",
    fan_in_fan_out=True,  # Setting fan_in_fan_out if it's a supported parameter
)

In [34]:
from transformers import BitsAndBytesConfig


q_config = BitsAndBytesConfig(
    load_in_4bit=True,
    q_4bit_quant_type="nf4",
    q_4bit_use_double_quant=True,
    q_4bit_compute_dtype=torch.bfloat16
)

In [35]:
# Update the GPT-2 model configuration to recognize the new padding token and the text task classification
config = GPT2Config.from_pretrained(
    "gpt2",
    num_labels=2,
    id2label={0: "NEGATIVE", 1: "POSITIVE"},
    label2id={"NEGATIVE": 0, "POSITIVE": 1},
    pad_token_id=tokenizer.eos_token_id  # Add pad token id to the model config
)

# Load the model with the updated configuration for sequence classification
q_model = AutoModelForSequenceClassification.from_pretrained("gpt2", config=config,quantization_config=q_config)


# Freeze all the parameters of the base model
for param in q_model.base_model.parameters():
    param.requires_grad = False

# The model is now ready with the appropriate settings for padding token
q_model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
from peft import get_peft_model
lora_model = get_peft_model(q_model, lora_config,"default")

In [37]:
tokenized_ds

{'train': Dataset({
     features: ['text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 500
 }),
 'test': Dataset({
     features: ['text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 500
 })}

In [38]:
tokenized_ds["train"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_ds["test"].set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

In [39]:
import torch
from torch.utils.data import DataLoader
from torch.optim import AdamW
from tqdm.auto import tqdm


In [40]:
train_dataloader = DataLoader(tokenized_ds["train"].select(range(100)), 
                              batch_size=8, 
                              collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))
eval_dataloader = DataLoader(tokenized_ds["test"].select(range(100)), 
                             batch_size=8, 
                             collate_fn=DataCollatorWithPadding(tokenizer=tokenizer))


In [41]:
optimizer = AdamW(lora_model.parameters(), lr=2e-4)

In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
lora_model.to(device)

for epoch in range(12):  # number of epochs
    lora_model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = lora_model(**batch)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
    
    print(f"Epoch {epoch} train loss: {train_loss / len(train_dataloader)}")

    # Evaluation
    lora_model.eval()
    eval_accuracy = 0
    with torch.no_grad():
        for batch in eval_dataloader:
            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = lora_model(**batch)
            predictions = torch.argmax(outputs.logits, dim=-1)
            eval_accuracy += (predictions == batch["labels"]).float().mean()

    eval_accuracy = eval_accuracy / len(eval_dataloader)
    print(f"Epoch {epoch} eval accuracy: {eval_accuracy}")


  0%|          | 0/13 [00:00<?, ?it/s]You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:224: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_type=torch.float32 (default). This will lead to slow inference or training speed.')
100%|██████████| 13/13 [00:07<00:00,  1.68it/s]


Epoch 0 train loss: 4.716947115384615
Epoch 0 eval accuracy: 0.5480769276618958


100%|██████████| 13/13 [00:07<00:00,  1.83it/s]


Epoch 1 train loss: 1.33349609375
Epoch 1 eval accuracy: 0.5384615659713745


100%|██████████| 13/13 [00:07<00:00,  1.81it/s]


...

Epoch 9 train loss: 0.3341815655048077
Epoch 9 eval accuracy: 0.634615421295166


100%|██████████| 13/13 [00:07<00:00,  1.69it/s]


Epoch 10 train loss: 0.2551069993239183
Epoch 10 eval accuracy: 0.625


100%|██████████| 13/13 [00:07<00:00,  1.71it/s]


Epoch 11 train loss: 0.1791053185096154
Epoch 11 eval accuracy: 0.6442307829856873


In [43]:
lora_model.print_trainable_parameters()

trainable params: 811,008 || all params: 125,252,352 || trainable%: 0.6474992182182735


In [45]:
#lora_model.save_pretrained("gpt-qlora")

## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [27]:
from transformers import GPT2ForSequenceClassification


In [31]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, AutoModelForSequenceClassification
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Correct checkpoint path
checkpoint_path = './data/sentiment_analysis/checkpoint-25'

# Load the model and assign it to a variable
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)

# Assuming 'tokenized_ds' and 'tokenizer' have been correctly defined earlier
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        # Specify a directory for saving training outputs
        output_dir='./data/sentiment_analysis/training_output',
        # Correct the lorategy='epoch',
        learning_rate=2e-4,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        num_train_epochs=10,
        weight_decay=0.01,
        save_strategy="epoch",
        logging_dir='./data/sentiment_analysis/logs',
    ),
    train_dataset=tokenized_ds["train"].select(range(100)),  # Using a subset for faster training
    eval_dataset=tokenized_ds["test"].select(range(100)),  # Using a subset for faster evaluation
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)


In [32]:
# Show the performance of the model on the test set
from transformers import Trainer
# What do you think the evaluation accuracy will be?
trainer.evaluate()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 0.6418538689613342,
 'eval_accuracy': 0.63,
 'eval_runtime': 360.6548,
 'eval_samples_per_second': 0.277,
 'eval_steps_per_second': 0.036}

## Lora Inference

In [35]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding, GPT2ForSequenceClassification

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"accuracy": (predictions == labels).mean()}

# Load the model from the correct checkpoint path
#checkpoint_path = r"C:\Users\Atilio\anaconda3\envs\envs\LangChain\LLMAppsUdemy\gpt-lora\"
checkpoint_path = './datapetflora/sentiment_analysis/checkpoint-9'


model = GPT2ForSequenceClassification.from_pretrained(checkpoint_path)
#model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
# Load the tokenizer from the checkpoint
tokenizer = GPT2TokenizerFast.from_pretrained(checkpoint_path)
#model.load_adapter("gpt-lora")

# Assuming 'tokenized_ds' and 'tokenizer' have been correctly defined earlier
trainer = Trainer(
    model=model,
    args=TrainingArguments(
        output_dir='./datapeftlora/sentiment_analysis/training_output',
        evaluation_strategy='epoch',
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        learning_rate=2e-4,
        num_train_epochs=10,
        weight_decay=0.01,
        save_strategy="epoch",
        load_best_model_at_end=True,
        logging_dir= './logs',
    ),
    train_dataset=tokenized_ds["train"].select(range(100)),  # Replace with your full training dataset
    eval_dataset=tokenized_ds["test"].select(range(100)),   # Replace with your full evaluation dataset
    tokenizer=tokenizer,  # Make sure this is correctly initialized
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer, padding=True),
    compute_metrics=compute_metrics,# Complete the data collator definition
)


Some weights of the model checkpoint at ./datapetflora/sentiment_analysis/checkpoint-9 were not used when initializing GPT2ForSequenceClassification: ['transformer.h.2.attn.c_proj.lora_A.default.weight', 'transformer.h.6.attn.c_proj.lora_B.default.weight', 'transformer.h.2.attn.c_attn.lora_B.default.weight', 'transformer.h.4.mlp.c_proj.lora_A.default.weight', 'transformer.h.11.mlp.c_proj.lora_A.default.weight', 'transformer.h.6.mlp.c_proj.lora_A.default.weight', 'transformer.h.2.mlp.c_proj.lora_A.default.weight', 'transformer.h.7.attn.c_attn.lora_A.default.weight', 'transformer.h.9.mlp.c_proj.lora_B.default.weight', 'transformer.h.7.attn.c_proj.lora_A.default.weight', 'transformer.h.1.attn.c_proj.lora_A.default.weight', 'transformer.h.10.mlp.c_proj.lora_A.default.weight', 'transformer.h.6.mlp.c_proj.lora_B.default.weight', 'transformer.h.0.mlp.c_proj.lora_A.default.weight', 'transformer.h.11.mlp.c_proj.lora_B.default.weight', 'transformer.h.8.attn.c_proj.lora_A.default.weight', 'transf

In [36]:
# Show the performance of the model on the test set
from transformers import Trainer
# What do you think the evaluation accuracy will be?
trainer.evaluate()# What do you think the evaluation accuracy will be?


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 5.193200588226318,
 'eval_accuracy': 0.47,
 'eval_runtime': 305.5018,
 'eval_samples_per_second': 0.327,
 'eval_steps_per_second': 0.043}

### Conclusions

In refining a sentiment analysis model with GPT-2 using LoRA and QLoRA, we faced challenges that limited our training scalability due to CUDA memory constraints, on the other hand we have never obtain during the training process better accuracy than 0.73 despite a consistent decrease in training loss. The issue appears to stem from improper model-saving practices, potentially saving early training states rather than optimized final states, suggesting a misalignment in model serialization between PyTorch and Transformers frameworks. This situation underscores the importance of precise model management and addressing hardware limitations to harness the full potential of advanced fine-tuning techniques.